# Modeling Notebook


## 1. Import libraries

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import os 
%config Completer.use_jedi = False

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
os.sys.path.append('../src')

## 2. Load Data

In [2]:
df_raw = pd.read_excel('../data/survey_answers.xlsx', sheet_name='Dados_brutos')
# df_raw = pd.read_csv('../data/survey-answers-no-outliers.txt', encoding = 'iso 8859-1', delimiter = '\t')
df_raw.head()

,Genero,Idade,Peso,Altura,IMC,Escolaridade,Estado_Civil,Filhos,Atividade_fisica_semana,Cigarros_dia,Horas_Dormidas_dia,Se_Capacitando,Outro_emprego,Ambiente_Trabalho,Categoria_Profissional,Cargo,Tempo_Empresa_anos,Tempo_Funcao_anos,Horas_Trabalhadas_Semana,Horas_Trabalhadas_Dia,Pausas_Programadas_minutos,Trabalho_Entre_Ferias_meses,Recebeu_Treinamento,Como_foi_treinado,Trabalho_Chato,Trabalho_Organizado,Prazos_Apertados,Satisfação_Gerencia,Satisfação_Dificuldades,Tempo_Pe_horas,Tempo_Sentado_horas,Tempo_Corcoras_horas,Superiores_Desconfortavel_horas,Inferiores_Desconfortavel_horas,Tronco_Curvado_horas,Tronco_Torcido_horas,Maos_Dedos_horas,Carga_6_horas,Carga_15_horas,Carga_25_horas,Movimentos_repetitivos_horas,Movimentos_Rapidos_horas,Ferramentas_Maos_horas,Ferramentas_Corpo_horas,Pescoco,Cervical,Costa_superior,Costa_media,Costa_inferior,Bacia,Ombro_esquerdo,Ombro_direito,Braco_esquerdo,Braco_direito,Cotovelo_esquerdo,Cotovelo_direito,Antebraco_esquerdo,Antebraço_direito,Mao_esquerda,Mao_direita,Coxa_esquerda,Coxa_direita,Joelho_esquerdo,Joelho_direito,Perna_esquerda,Perna_direita,Tornozelo_esquerdo,Tornozelo_direito,Pe_esquerdo,Pe_direito,Dedos_direito,Polegar_direito,Interfalange_direito,Palma_mao_direito,Punho_direito,Radial_dist_direito,Ulnar_dist_direito,Radial_prox_direito,Ulnar_prox_direito,Dedos_esquerdo,Polegar_esquerdo,Interfalange_esquerdo,Palma_mao_esquerdo,Punho_esquerdo,Radial_dist_esquerdo,Ulnar_dist_esquerdo,Radial_prox_esquerdo,Ulnar_prox_esquerdo,Q1,Q2,Q3,Q4,Q5,Q6,Q7,Q8,Q9,Q10,Q11,Q12,Q13,Q14,Q15,Q16,Q17,Q18,Q19,Q20,Q21,Q22,Q23,Q24,Q25,Q26,Q27,Q28,Q29,Q30,Q31,Q32,Q33,Q34,Q35,Q36,Q37,Q38,Q39,Q40,Q41,Q42,Q43,Q44,Q45,Q46,Q47,Q48,Q49,Q50,Q51,Q52,Q53,Q54,Q55,Q56,Q57,Q58,Q59,Q60,Q61,Q62,Q63,Q64,Q65,Q66,Q67,Q68,Q69,Q70,Q71,Q72
0,1,36,61.0,1.67,21.872423,3,2,2,0,0,6,2,2,2,1,Tecnico_de_enfermagem,8.0,3.0,36,12.0,30,11,2,1,1,5,5,5,4,11.0,0.5,0.0,0.0,0.0,1.0,0.0,12.0,1.0,1.0,1.0,12.0,12.0,1.0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,3,1,2,1,5,5,2,5,5,5,5,3,5,5,5,4,5,5,3,5,5,5,4,2,2,5,4,3,4,5,2,2,3,2,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,2,3,3,4,4,4,2,2,3,3,4,3,4,4,3,2,4,4,3,4,2,4,2
1,1,30,65.0,1.62,24.767566,3,2,1,5,0,6,1,2,2,1,Tecnico_de_enfermagem,4.0,7.0,40,8.0,120,11,1,5,1,4,1,4,3,4.0,4.0,0.0,0.0,0.0,0.0,0.0,8.0,0.0,0.0,0.0,8.0,0.0,0.0,0,1,1,1,2,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,2,1,1,1,2,1,1,1,1,2,1,1,1,2,1,1,1,1,1,1,3,1,4,5,4,1,5,5,5,2,4,5,4,3,4,5,1,5,5,5,5,5,4,2,2,2,1,1,2,4,3,3,2,4,4,2,3,4,4,4,3,5,4,4,2,2,5,2,2,2,5,5,5,2,3,3,2,2,2,3,2,4,5,3,5,5,2,2,2,5
2,1,40,79.0,1.51,34.647603,3,2,3,0,0,10,1,2,1,1,Tecnico_de_enfermagem,8.0,12.0,36,12.0,60,11,1,4,1,5,1,4,3,6.0,6.0,0.0,0.0,0.0,0.0,0.0,12.0,0.0,0.0,0.0,12.0,3.0,0.0,0,1,1,2,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,3,5,5,5,5,5,5,5,5,2,5,2,4,5,5,2,5,5,5,5,2,2,3,2,5,2,2,3,2,2,3,2,4,4,1,4,4,4,5,4,5,5,5,1,1,5,1,2,2,5,5,5,1,1,3,2,1,5,2,2,1,4,3,4,5,2,2,1,5
3,1,45,76.0,1.60,29.687500,3,2,1,3,0,8,1,2,1,1,Tecnico_de_enfermagem,8.0,18.0,36,12.0,60,11,1,5,2,5,4,4,3,6.0,6.0,0.0,4.0,8.0,2.0,0.0,12.0,0.0,0.0,0.0,12.0,3.0,0.0,0,3,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,2,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,3,5,5,5,5,5,5,5,4,3,5,4,4,5,5,3,5,5,5,5,2,2,2,2,1,2,2,3,2,2,2,5,4,4,2,5,4,4,4,5,5,4,4,1,1,1,1,1,1,4,5,4,1,3,4,1,2,3,2,3,2,3,4,4,4,1,1,1,3
4,1,41,67.0,1.68,23.738662,5,1,2,5,0,7,2,2,1,1,Tecnico_de_enfermagem,4.0,4.0,36,12.0,0,11,1,3,1,5,3,4,4,11.0,1.0,1.0,10.0,10.0,0.0,0.0,12.0,1.0,1.0,1.0,12.0,10.0,0.0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,5,5,1,1,1,1,1,2,1,1,5,1,1,1,1,1,2,1,1,5,1,1,1,1,3,1,1,1,5,5,5,5,5,5,5,1,3,3,5,5,4,5,4,5,4,5,5,2,2,4,4,5,4,3,4,2,4,4,4,4,4,2,4,4,5,3,3,3,3,5,3,3,4,2,2,2,3,4,4,2,4,2,3,1,3,3,1,4,3,4,5,5,2,2,3,4


## 3. Feature Engineering

In [3]:
df_analysis = df_raw.copy()
df_analysis['IMC'] = df_analysis['Peso']/df_analysis['Altura']**2

In [4]:
def bmi_category(bmi_value):
    if bmi_value < 18:
        return 'underweight'
    elif bmi_value >= 18 and bmi_value <= 24:
        return 'normal'
    elif bmi_value > 24 and bmi_value <= 28:
        return 'overweight'
    elif bmi_value > 28:
        return 'obese'
    else:
        return 'error'

In [5]:
df_analysis.insert(loc = 1, column = 'IMC_categoria', value = df_analysis['IMC'].apply(bmi_category))

In [6]:
df_analysis.insert(loc = 1, 
                   column = 'Horas_Carregando_Carga', 
                   value = df_analysis['Carga_6_horas'] + df_analysis['Carga_15_horas'] + df_analysis['Carga_25_horas'])

## 4. Target definition

In [7]:
def target_grouping(score):
    if score >= 4:
        return 'severe pain'
    elif score in [2,3]:
        return 'mild pain'
    else:
        return 'no pain'

In [8]:
df_analysis['Costa_superior_agg'] = df_analysis['Costa_superior'].apply(target_grouping)
df_analysis['Costa_media_agg'] = df_analysis['Costa_media'].apply(target_grouping)
df_analysis['Costa_inferior_agg'] = df_analysis['Costa_inferior'].apply(target_grouping)

## 5. Feature Selection

We'll fit a Random Forest model and will use its feature importance values to select the most relevant features.

- [Article](https://johaupt.github.io/blog/columnTransformer_feature_names.html) with wraper function to get feature names

In [52]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import LogisticRegression

preprocessing = experiments.first_preprocessing
model = DecisionTreeClassifier()

sel = SelectFromModel(model)
target_column = 'Costa_superior_agg'
X_train_preprocessed = preprocessing.fit_transform(X_train)
sel = model.fit(X_train_preprocessed, y_train[target_column])

In [64]:
import sklearn
import warnings
def get_feature_names(column_transformer):
    """Get feature names from all transformers.
    Returns
    -------
    feature_names : list of strings
        Names of the features produced by transform.
    """
    # Remove the internal helper function
    #check_is_fitted(column_transformer)
    
    # Turn loopkup into function for better handling with pipeline later
    def get_names(trans):
        # >> Original get_feature_names() method
        if trans == 'drop' or (
                hasattr(column, '__len__') and not len(column)):
            return []
        if trans == 'passthrough':
            if hasattr(column_transformer, '_df_columns'):
                if ((not isinstance(column, slice))
                        and all(isinstance(col, str) for col in column)):
                    return column
                else:
                    return column_transformer._df_columns[column]
            else:
                indices = np.arange(column_transformer._n_features)
                return ['x%d' % i for i in indices[column]]
        if not hasattr(trans, 'get_feature_names'):
        # >>> Change: Return input column names if no method avaiable
            # Turn error into a warning
            warnings.warn("Transformer %s (type %s) does not "
                                 "provide get_feature_names. "
                                 "Will return input column names if available"
                                 % (str(name), type(trans).__name__))
            # For transformers without a get_features_names method, use the input
            # names to the column transformer
            if column is None:
                return []
            else:
                return [name + "__" + f for f in column]

        return [name + "__" + f for f in trans.get_feature_names()]
    
    ### Start of processing
    feature_names = []
    
    # Allow transformers to be pipelines. Pipeline steps are named differently, so preprocessing is needed
    if type(column_transformer) == sklearn.pipeline.Pipeline:
        l_transformers = [(name, trans, None, None) for step, name, trans in column_transformer._iter()]
    else:
        # For column transformers, follow the original method
        l_transformers = list(column_transformer._iter(fitted=True))
    
    
    for name, trans, column, _ in l_transformers: 
        if type(trans) == sklearn.pipeline.Pipeline:
            # Recursive call on pipeline
            _names = get_feature_names(trans)
            # if pipeline has no transformer that returns names
            if len(_names)==0:
                _names = [name + "__" + f for f in column]
            feature_names.extend(_names)
        else:
            feature_names.extend(get_names(trans))
    
    return feature_names

In [67]:
feature_importance_df = pd.DataFrame({'feature_name': get_feature_names(preprocessing),
                                      'feature_importance': sel.feature_importances_
                                     })
feature_importance_df.sort_values(by =['feature_importance'], ascending = False)

/var/folders/b7/bgdywzjn66x2_2jj0sl00131cxdxqv/T/ipykernel_79346/733556529.py:32: UserWarning: Transformer imputer (type SimpleImputer) does not provide get_feature_names. Will return input column names if available
  warnings.warn("Transformer %s (type %s) does not "
/var/folders/b7/bgdywzjn66x2_2jj0sl00131cxdxqv/T/ipykernel_79346/733556529.py:32: UserWarning: Transformer scaler (type StandardScaler) does not provide get_feature_names. Will return input column names if available
  warnings.warn("Transformer %s (type %s) does not "


,feature_name,feature_importance
3,num__Tempo_Empresa_anos,0.127641
10,num__Pausas_Programadas_minutos,0.083221
15,num__Maos_Dedos_horas,0.078345
19,num__Movimentos_repetitivos_horas,0.070204
12,num__Inferiores_Desconfortavel_horas,0.064639
9,num__Horas_Dormidas_dia,0.064554
5,num__Tempo_Pe_horas,0.052971
6,num__Tempo_Sentado_horas,0.052014
14,num__Tronco_Torcido_horas,0.051714
2,num__Horas_Trabalhadas_Semana,0.043640


## 6. Train and test split
We'll split the data into train and test set. The train set will be used to perform 10-fold cross validation. We'll compute metrics for the test set and compare them.

In [9]:
print(df_raw.columns)

Index(['Genero', 'Idade', 'Peso', 'Altura', 'IMC', 'Escolaridade',
       'Estado_Civil', 'Filhos', 'Atividade_fisica_semana', 'Cigarros_dia',
       ...
       'Q63', 'Q64', 'Q65', 'Q66', 'Q67', 'Q68', 'Q69', 'Q70', 'Q71', 'Q72'],
      dtype='object', length=160)


In [10]:
from sklearn.model_selection import train_test_split
import experiments
numerical_features = experiments.numerical_features
categorical_features = experiments.categorical_features
target_columns = ['Costa_superior_agg', 'Costa_media_agg', 'Costa_inferior_agg']
# target_columns = ['dados.Costa_superior', 'dados.Costa_media', 'dados.Costa_inferior']

X = df_analysis[numerical_features + categorical_features]
y = df_analysis[target_columns]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42)

## 7. Experiments

- [Article](https://towardsdatascience.com/quickly-test-multiple-models-a98477476f0) used as reference to wrap code.

### 7.1. Upper Back

In [11]:
scorer = experiments.scorers
algorithms = experiments.build_algorithms()
cv = experiments.cv
target_column = 'Costa_superior_agg'

In [12]:
from sklearn.metrics import classification_report
from sklearn.model_selection import cross_val_score, cross_validate

def run_exps(X_train: pd.DataFrame , y_train: pd.DataFrame, X_test: pd.DataFrame, y_test: pd.DataFrame) -> pd.DataFrame:
    '''
    Lightweight script to test many models and find winners
    :param X_train: training split
    :param y_train: training target vector
    :param X_test: test split
    :param y_test: test target vector
    :return: DataFrame of predictions
    '''
    
    dfs = []

    # results = []
    names = []
    scoring = ['accuracy', 'precision_macro', 'recall_macro', 'f1_macro']
    target_names = ['no pain', 'severe pain', 'mild pain']
    for name, model in algorithms.items():
        kfold = cv#StratifiedKFold(n_splits=3, random_state=42, shuffle=True)
        cv_results = cross_validate(model, X_train, y_train, cv=kfold, scoring=scoring)
        clf = model.fit(X_train, y_train)
        y_pred = clf.predict(X_test)
        print(name)
        print(classification_report(y_test, y_pred, target_names=target_names))
        # results.append(cv_results)
        names.append(name)
        this_df = pd.DataFrame(cv_results)
        this_df['model'] = name
        dfs.append(this_df)
    final = pd.concat(dfs, ignore_index=True)
    return final

In [13]:
run_exps(X_train, y_train[target_column], X_test, y_test[target_column])

kNN
              precision    recall  f1-score   support

     no pain       0.36      0.47      0.40        45
 severe pain       0.48      0.48      0.48        61
   mild pain       0.33      0.10      0.15        20

    accuracy                           0.41       126
   macro avg       0.39      0.35      0.34       126
weighted avg       0.41      0.41      0.40       126

tree
              precision    recall  f1-score   support

     no pain       0.42      0.62      0.50        45
 severe pain       0.56      0.39      0.46        61
   mild pain       0.29      0.25      0.27        20

    accuracy                           0.45       126
   macro avg       0.43      0.42      0.41       126
weighted avg       0.47      0.45      0.45       126



/Users/56325/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/56325/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/56325/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


svmlinear
              precision    recall  f1-score   support

     no pain       0.42      0.76      0.54        45
 severe pain       0.56      0.41      0.47        61
   mild pain       0.00      0.00      0.00        20

    accuracy                           0.47       126
   macro avg       0.33      0.39      0.34       126
weighted avg       0.42      0.47      0.42       126



/Users/56325/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/56325/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/56325/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/56325/opt/an

ann
              precision    recall  f1-score   support

     no pain       0.37      0.47      0.41        45
 severe pain       0.50      0.46      0.48        61
   mild pain       0.23      0.15      0.18        20

    accuracy                           0.41       126
   macro avg       0.37      0.36      0.36       126
weighted avg       0.41      0.41      0.41       126



/Users/56325/opt/anaconda3/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(


,fit_time,score_time,test_accuracy,test_precision_macro,test_recall_macro,test_f1_macro,model
0,0.481240,0.008600,0.489796,0.659259,0.417460,0.416467,kNN
1,0.454869,0.008594,0.489796,0.444444,0.413589,0.413806,kNN
2,0.466609,0.008470,0.387755,0.342409,0.346767,0.342299,kNN
3,0.468351,0.005309,0.418367,0.361902,0.359127,0.359472,tree
4,0.419572,0.006031,0.520408,0.372354,0.410763,0.378898,tree
5,0.412135,0.005344,0.397959,0.361598,0.341580,0.348480,tree
6,0.523319,0.010449,0.438776,0.296784,0.351190,0.316667,svmlinear
7,0.510622,0.006769,0.510204,0.341300,0.401858,0.368305,svmlinear
8,0.509071,0.006655,0.438776,0.292363,0.345141,0.316377,svmlinear
9,1.916649,0.007899,0.459184,0.396128,0.391667,0.388673,ann


### 7.2. Mid Back

In [14]:
target_column = 'Costa_media_agg'
run_exps(X_train, y_train[target_column], X_test, y_test[target_column])

kNN
              precision    recall  f1-score   support

     no pain       0.49      0.50      0.50        50
 severe pain       0.50      0.60      0.55        55
   mild pain       0.33      0.14      0.20        21

    accuracy                           0.48       126
   macro avg       0.44      0.41      0.41       126
weighted avg       0.47      0.48      0.47       126

tree
              precision    recall  f1-score   support

     no pain       0.38      0.30      0.33        50
 severe pain       0.46      0.53      0.49        55
   mild pain       0.26      0.29      0.27        21

    accuracy                           0.40       126
   macro avg       0.37      0.37      0.37       126
weighted avg       0.39      0.40      0.39       126



/Users/56325/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/56325/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/56325/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


svmlinear
              precision    recall  f1-score   support

     no pain       0.40      0.08      0.13        50
 severe pain       0.45      0.95      0.61        55
   mild pain       0.00      0.00      0.00        21

    accuracy                           0.44       126
   macro avg       0.28      0.34      0.25       126
weighted avg       0.35      0.44      0.32       126



/Users/56325/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/56325/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/56325/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/56325/opt/an

ann
              precision    recall  f1-score   support

     no pain       0.47      0.44      0.45        50
 severe pain       0.52      0.56      0.54        55
   mild pain       0.16      0.14      0.15        21

    accuracy                           0.44       126
   macro avg       0.38      0.38      0.38       126
weighted avg       0.44      0.44      0.44       126



/Users/56325/opt/anaconda3/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(


,fit_time,score_time,test_accuracy,test_precision_macro,test_recall_macro,test_f1_macro,model
0,0.548207,0.009578,0.500000,0.438126,0.439805,0.435719,kNN
1,0.513804,0.008635,0.479592,0.316366,0.372239,0.341295,kNN
2,0.443665,0.009794,0.408163,0.370130,0.368861,0.366126,kNN
3,0.524584,0.006635,0.438776,0.415302,0.432763,0.419394,tree
4,0.484598,0.005855,0.346939,0.313941,0.316401,0.314552,tree
5,0.487049,0.006789,0.520408,0.678075,0.445048,0.441115,tree
6,0.601327,0.008773,0.448980,0.154386,0.312057,0.206573,svmlinear
7,0.594668,0.008965,0.438776,0.245515,0.314691,0.253333,svmlinear
8,0.537195,0.007237,0.500000,0.427957,0.363124,0.280868,svmlinear
9,2.087895,0.008805,0.489796,0.451634,0.454483,0.451824,ann


### 7.3. Low Back

In [15]:
target_column = 'Costa_inferior_agg'
run_exps(X_train, y_train[target_column], X_test, y_test[target_column])

kNN
              precision    recall  f1-score   support

     no pain       0.62      0.55      0.58        53
 severe pain       0.57      0.54      0.55        54
   mild pain       0.21      0.32      0.26        19

    accuracy                           0.51       126
   macro avg       0.47      0.47      0.46       126
weighted avg       0.54      0.51      0.52       126

tree
              precision    recall  f1-score   support

     no pain       0.52      0.58      0.55        53
 severe pain       0.51      0.44      0.48        54
   mild pain       0.16      0.16      0.16        19

    accuracy                           0.46       126
   macro avg       0.40      0.40      0.39       126
weighted avg       0.46      0.46      0.46       126



/Users/56325/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


svmlinear
              precision    recall  f1-score   support

     no pain       0.59      0.45      0.51        53
 severe pain       0.48      0.76      0.59        54
   mild pain       0.00      0.00      0.00        19

    accuracy                           0.52       126
   macro avg       0.36      0.40      0.37       126
weighted avg       0.45      0.52      0.47       126



/Users/56325/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/56325/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/56325/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/56325/opt/an

ann
              precision    recall  f1-score   support

     no pain       0.49      0.47      0.48        53
 severe pain       0.48      0.46      0.47        54
   mild pain       0.17      0.21      0.19        19

    accuracy                           0.43       126
   macro avg       0.38      0.38      0.38       126
weighted avg       0.44      0.43      0.43       126



/Users/56325/opt/anaconda3/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(


,fit_time,score_time,test_accuracy,test_precision_macro,test_recall_macro,test_f1_macro,model
0,0.503466,0.008162,0.367347,0.350052,0.356226,0.352073,kNN
1,0.467142,0.008593,0.346939,0.326840,0.330552,0.327796,kNN
2,0.472925,0.008020,0.397959,0.388057,0.392251,0.389173,kNN
3,0.411642,0.005086,0.336735,0.392862,0.360719,0.337999,tree
4,0.385016,0.004910,0.316327,0.261757,0.269148,0.265029,tree
5,0.405631,0.005211,0.448980,0.437566,0.434559,0.434976,tree
6,0.528778,0.006391,0.469388,0.428266,0.395593,0.373499,svmlinear
7,0.513533,0.006015,0.438776,0.418395,0.385537,0.376516,svmlinear
8,0.489586,0.006784,0.469388,0.308846,0.386111,0.329286,svmlinear
9,1.955193,0.006989,0.397959,0.353801,0.354942,0.354308,ann


## 8. Feature Analysis